<a href="https://colab.research.google.com/github/zzh8829/yolov3-tf2/blob/master/colab_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YoloV3 TF2 GPU Colab Notebook

##### 1.  Clone and install dependencies 

**IMPORTANT**: Restart following the instruction

In [ ]:
#!git clone https://github.com/zzh8829/yolov3-tf2
#%cd yolov3-tf2/
#!pip install -r requirements-gpu.txt

##### 2.  Check Tensorflow2 version

In [3]:
%cd yolov3-tf2/
!ls

import tensorflow as tf
tf.__version__

[Errno 2] No such file or directory: 'yolov3-tf2/'
/home/pohjao/Source/yolov3-tf2-auair
checkpoints	 data		  docs	     requirements-gpu.txt  tools
colab_gpu.ipynb  detect.py	  LICENSE    requirements.txt	   train.py
convert.py	 detect_video.py  README.md  setup.py		   yolov3_tf2


'2.2.0'

##### 3.  Convert Pretrained Darknet Weight

In [5]:
!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights
!python convert.py

--2020-05-29 11:30:09--  https://pjreddie.com/media/files/yolov3.weights
Selvitetään osoitetta pjreddie.com (pjreddie.com)... 128.208.4.108
Yhdistetään palvelimeen pjreddie.com (pjreddie.com)|128.208.4.108|:443... yhdistetty.
HTTP-pyyntö lähetetty, odotetaan vastausta... 200 OK
Pituus: 248007048 (237M) [application/octet-stream]
Tallennetaan kohteeseen ”data/yolov3.weights”

data/yolov3.weights 100%[===================>] 236,52M   105KB/s    in 49m 6s  

2020-05-29 12:19:18 (82,2 KB/s) - ”data/yolov3.weights” tallennettu [248007048/248007048]

2020-05-29 12:19:19.764226: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-29 12:19:19.769441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-29 12:19:19.769778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Foun

I0529 12:19:23.564717 140677534705472 utils.py:44] yolo_darknet/conv2d_20 bn
I0529 12:19:23.566517 140677534705472 utils.py:44] yolo_darknet/conv2d_21 bn
I0529 12:19:23.569135 140677534705472 utils.py:44] yolo_darknet/conv2d_22 bn
I0529 12:19:23.571105 140677534705472 utils.py:44] yolo_darknet/conv2d_23 bn
I0529 12:19:23.573911 140677534705472 utils.py:44] yolo_darknet/conv2d_24 bn
I0529 12:19:23.575590 140677534705472 utils.py:44] yolo_darknet/conv2d_25 bn
I0529 12:19:23.578216 140677534705472 utils.py:44] yolo_darknet/conv2d_26 bn
I0529 12:19:23.598922 140677534705472 utils.py:44] yolo_darknet/conv2d_27 bn
I0529 12:19:23.601349 140677534705472 utils.py:44] yolo_darknet/conv2d_28 bn
I0529 12:19:23.608920 140677534705472 utils.py:44] yolo_darknet/conv2d_29 bn
I0529 12:19:23.611603 140677534705472 utils.py:44] yolo_darknet/conv2d_30 bn
I0529 12:19:23.618819 140677534705472 utils.py:44] yolo_darknet/conv2d_31 bn
I0529 12:19:23.621109 140677534705472 utils.py:44] yolo_darknet/conv2d_32 bn

##### 4. Train with the AU-AIR dataset

In [ ]:
#Download AU-AIR data and run tfrecord-conversion

In [4]:
!python tools/visualize_dataset.py --dataset ./data/auair_train.tfrecord --classes=./data/auair.names

Traceback (most recent call last):
  File "tools/visualize_dataset.py", line 7, in <module>
    from yolov3_tf2.models import (
ModuleNotFoundError: No module named 'yolov3_tf2'


In [ ]:
from IPython.display import Image
Image(filename='./output.jpg')

In [ ]:
!python train.py \
	--dataset ./data/voc_train.tfrecord \
	--val_dataset ./data/voc_val.tfrecord \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--mode fit --transfer darknet \
	--batch_size 16 \
	--epochs 3 \
	--weights ./checkpoints/yolov3.tf \
	--weights_num_classes 80 

##### 4. Initialize Detector

In [ ]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

flags.DEFINE_string('classes', './data/auair.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/parking_lot.jpg', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 8, 'number of classes in the model')

app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

##### 4. Detect Image

In [ ]:
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)
      
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))

img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

##### 6. Detect using new weights

In [ ]:
FLAGS.num_classes = 20
FLAGS.classes = 'data/voc2012.names'
FLAGS.weights = 'checkpoints/yolov3_train_3.tf'
FLAGS.image = 'data/meme.jpg'

# Lower threshold due to insufficient training
FLAGS.yolo_iou_threshold = 0.2
FLAGS.yolo_score_threshold = 0.2

if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))

img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))